In [ ]:
top = 1

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import kurtosis, skew
from itertools import combinations
import minepy
from collections import Counter
from sklearn.metrics import mutual_info_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)   # 5-fold-cross validation

In [ ]:
import bayesiantests as bt
import matplotlib.pyplot as plt

#https://matplotlib.org/stable/gallery/lines_bars_and_markers/horizontal_barchart_distribution.html
def stacked_bar(results, category_names):
    """
    Parameters
    ----------
    results : dict
        A mapping from question labels to a list of answers per category.
        It is assumed all lists contain the same number of entries and that
        it matches the length of *category_names*.
    category_names : list of str
        The category labels.
    """
    labels = list(results.keys())
    data = np.array(list(results.values()))
    data_cum = data.cumsum(axis=1)
    category_colors = plt.colormaps['RdYlGn'](
        np.linspace(0.15, 0.85, data.shape[1]))

    fig, ax = plt.subplots(figsize=(9.2, 5))
    ax.invert_yaxis()
    ax.xaxis.set_visible(False)
    ax.set_xlim(0, np.sum(data, axis=1).max())

    for i, (colname, color) in enumerate(zip(category_names, category_colors)):
        widths = data[:, i]
        starts = data_cum[:, i] - widths
        rects = ax.barh(labels, widths, left=starts, height=0.5,
                        label=colname, color=color)

        r, g, b, _ = color
        text_color = 'white' if r * g * b < 0.5 else 'darkgrey'
        ax.bar_label(rects, label_type='center', color=text_color)
    ax.legend(ncols=len(category_names), bbox_to_anchor=(0, 1),
              loc='upper left', fontsize='small')

    return fig, ax

In [ ]:
def BST(rope, baselines, ours, dfs):
    comp = []
    basewin = []
    draw = []
    ourswin = []
    z = 0
    for i in range(len(ours)):
        for j in range(len(baselines)):
            names = (baselines[j],ours[i])
            comp.append(names)
            X = np.array(dfs[i][[baselines[j],ours[i]]])
            left, within, right = bt.signtest(X, rope=rope, verbose=True, names=names)
            basewin.append(left)
            draw.append(within)
            ourswin.append(right)        
    results = pd.DataFrame(comp, columns = ["Baseline","Ours"])
    results["Basewin_prob"] = basewin
    results["Draw_prob"] = draw
    results["Ourswin_prob"] = ourswin
    return results

In [ ]:
final_list = [18, 41, 14, 43, 53, 28, 20, 63, 69, 56, 19, 25, 6, 24, 80, 32, 22, 15, 27, 33, 58,
              46, 29, 64, 62, 17, 47, 13, 44, 9, 49, 55, 3, 35, 67, 54, 12, 7, 39, 36, 4, 79, 59, 52, 5, 57,
              21, 50, 45, 42, 11, 1, 51, 38, 34, 16, 10, 2, 26, 91]
print(len(final_list), final_list)

In [ ]:
df_meta_loaded = pd.read_csv("df_meta_new.csv", index_col='Unnamed: 0')
df_meta_loaded

In [ ]:
new_index = []
for i in range(len(df_meta_loaded)):
    if list(df_meta_loaded['data'])[i] in final_list:
        new_index.append(i)
df_meta = df_meta_loaded.iloc[new_index,:]
df_meta.reset_index(inplace=True, drop=True)
df_meta

# Analysis

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('result.csv', low_memory=False)
df.rename(columns={'Unnamed: 0':'Metrics'}, inplace=True)
df

In [ ]:
df_LS = pd.read_csv('result_LS.csv', low_memory=False)
df_LS.rename(columns={'Unnamed: 0':'Metrics'}, inplace=True)
df_LS

In [ ]:
# F1
base_ind = 2
num_class = (df.shape[1]-base_ind)/(1+5+5)
org_ind = int(base_ind+num_class)
sm_ind = int(org_ind+(df.shape[1]-base_ind-num_class)/2)
metrics = ['Acc', 'Pre', 'Rec', 'Spe', 'F1', 'GM', 'BA', 'AUC'] # 8:Acc, 9:Pre, 10:Rec, 11:Spe, 12:F1, 13:GM, 14:BA, 15:AUC 
df_res = pd.DataFrame({'DATA':[0 for i in range(top*len(final_list))],
                       'ORG_tr':[0 for i in range(top*len(final_list))],
                       'ORG_va':[0 for i in range(top*len(final_list))],
                       'ORG_t':[0 for i in range(top*len(final_list))],
                       'SMOTE_tr':[0 for i in range(top*len(final_list))],
                       'SMOTE_va':[0 for i in range(top*len(final_list))],
                       'SMOTE_t':[0 for i in range(top*len(final_list))],
                       'LLM_tr':[0 for i in range(top*len(final_list))],
                       'LLM_va':[0 for i in range(top*len(final_list))],
                       'LLM_t':[0 for i in range(top*len(final_list))]})

for i in range(len(final_list)):        # in one dataset
    for j in range(len(metrics)*2):       # in one metric
        if j == 12:                      # only choose one metric (12:validation_F1)
            df_i = df[df["Dataset"] == str(final_list[i])]   # i_th dataset
            df_i_base = df_i.iloc[:,:base_ind]               # i_th dataset base
            df_i_org = df_i.iloc[:,base_ind:org_ind]         # i_th dataset original results (12 classifiers)
            df_i_sm = df_i.iloc[:,org_ind:sm_ind]            # i_th dataset smote results (12 classifier X 5 Resam strategy)
            df_i_lm = df_i.iloc[:,sm_ind:]                   # i_th dataset llm results (12 classifier X 5 Resam strategy)  
            best_org = pd.DataFrame(df_i_org.iloc[j,:]).iloc[:,0].sort_values(ascending=False)[:top].index
            df_i_org_best = df_i_org.loc[:,best_org] 
            best_sm = pd.DataFrame(df_i_sm.iloc[j,:]).iloc[:,0].sort_values(ascending=False)[:top].index
            df_i_sm_best = df_i_sm.loc[:,best_sm]
            best_lm = pd.DataFrame(df_i_lm.iloc[j,:]).iloc[:,0].sort_values(ascending=False)[:top].index
            df_i_lm_best = df_i_lm.loc[:,best_lm]
            df_i_best = pd.concat([df_i_base,df_i_org_best],axis=1)
            df_i_best = pd.concat([df_i_best,df_i_sm_best],axis=1)
            df_i_best = pd.concat([df_i_best,df_i_lm_best],axis=1)
            for k in range(top):
                df_res.iloc[i*top+k:i*top+k+1,0] = final_list[i]
                df_res.iloc[i*top+k:i*top+k+1,1] = list(df_i_best.iloc[[j-8],2+k])
                df_res.iloc[i*top+k:i*top+k+1,2] = list(df_i_best.iloc[[j],2+k])
                df_res.iloc[i*top+k:i*top+k+1,3] = list(df_i_best.iloc[[j+8],2+k])
                df_res.iloc[i*top+k:i*top+k+1,4] = list(df_i_best.iloc[[j-8],2+top+k])
                df_res.iloc[i*top+k:i*top+k+1,5] = list(df_i_best.iloc[[j],2+top+k])
                df_res.iloc[i*top+k:i*top+k+1,6] = list(df_i_best.iloc[[j+8],2+top+k])
                df_res.iloc[i*top+k:i*top+k+1,7] = list(df_i_best.iloc[[j-8],2+top+top+k])
                df_res.iloc[i*top+k:i*top+k+1,8] = list(df_i_best.iloc[[j],2+top+top+k])
                df_res.iloc[i*top+k:i*top+k+1,9] = list(df_i_best.iloc[[j+8],2+top+top+k])

In [ ]:
# F1-LS
df_res_LS = pd.DataFrame({'LS_tr':[0 for i in range(top*len(final_list))],
                          'LS_va':[0 for i in range(top*len(final_list))],
                          'LS_t':[0 for i in range(top*len(final_list))]})
for i in range(len(final_list)):        # in one dataset
    for j in range(len(metrics)*2):       # in one metric
        if j == 12:                      # only choose one metric (12:validation_F1)
            df_i = df_LS[df_LS["Dataset"] == str(final_list[i])]   # i_th dataset
            df_i_base = df_i.iloc[:,:base_ind]               # i_th dataset base
            df_i_LS = df_i.iloc[:,base_ind:]                 # i_th dataset LS results 
            best_LS = pd.DataFrame(df_i_LS.iloc[j,:]).iloc[:,0].sort_values(ascending=False)[:top].index
            df_i_LS_best = df_i_LS.loc[:,best_LS]     
            df_i_best = pd.concat([df_i_base,df_i_LS_best],axis=1)
            for k in range(top):
                df_res_LS.iloc[i*top+k:i*top+k+1,0] = list(df_i_best.iloc[[j-8],2+k])
                df_res_LS.iloc[i*top+k:i*top+k+1,1] = list(df_i_best.iloc[[j],2+k])
                df_res_LS.iloc[i*top+k:i*top+k+1,2] = list(df_i_best.iloc[[j+8],2+k])

In [ ]:
df_res_final = pd.concat([df_res, df_res_LS], axis=1)

In [ ]:
df_best_scores = df_res_final
df_best_scores

In [ ]:
# Averaged Results
df_best_scores_avg = pd.DataFrame(final_list, columns=['DATA'])
for i in range(1, df_best_scores.shape[1]):
    avg_list = []
    for j in range(len(df_best_scores_avg)):
        avg_list.append(np.average(df_best_scores.iloc[j*top:j*top+top,i]))
    df_best_scores_avg[f'{df_best_scores.columns[i]}'] = avg_list
df_best_scores_avg

In [ ]:
# Performance Margin (val)
df_best_scores_avg['(LM-SM)_va'] = df_best_scores_avg.loc[:,'LLM_va'].astype(float) - df_best_scores_avg.loc[:,'SMOTE_va'].astype(float)
df_best_scores_avg['(LS-LM)_va'] = df_best_scores_avg.loc[:,'LS_va'].astype(float) - df_best_scores_avg.loc[:,'LLM_va'].astype(float)
df_best_scores_avg['(LS-SM)_va'] = df_best_scores_avg.loc[:,'LS_va'].astype(float) - df_best_scores_avg.loc[:,'SMOTE_va'].astype(float)
# Performance Margin (test)
df_best_scores_avg['(LM-SM)_t'] = df_best_scores_avg.loc[:,'LLM_t'].astype(float) - df_best_scores_avg.loc[:,'SMOTE_t'].astype(float)
df_best_scores_avg['(LS-LM)_t'] = df_best_scores_avg.loc[:,'LS_t'].astype(float) - df_best_scores_avg.loc[:,'LLM_t'].astype(float)
df_best_scores_avg['(LS-SM)_t'] = df_best_scores_avg.loc[:,'LS_t'].astype(float) - df_best_scores_avg.loc[:,'SMOTE_t'].astype(float)

# Decision (val)
df_best_scores_avg['(LM>SM)_va'] = [1 if df_best_scores_avg.loc[i,"LLM_va"] > df_best_scores_avg.loc[i,"SMOTE_va"] else 0 for i in range(len(df_best_scores_avg))]
df_best_scores_avg['(LM=SM)_va'] = [1 if df_best_scores_avg.loc[i,"LLM_va"] == df_best_scores_avg.loc[i,"SMOTE_va"] else 0 for i in range(len(df_best_scores_avg))]
df_best_scores_avg['(LS>LM)_va'] = [1 if df_best_scores_avg.loc[i,"LS_va"] > df_best_scores_avg.loc[i,"LLM_va"] else 0 for i in range(len(df_best_scores_avg))]
df_best_scores_avg['(LS=LM)_va'] = [1 if df_best_scores_avg.loc[i,"LS_va"] == df_best_scores_avg.loc[i,"LLM_va"] else 0 for i in range(len(df_best_scores_avg))]
df_best_scores_avg['(LS>SM)_va'] = [1 if df_best_scores_avg.loc[i,"LS_va"] > df_best_scores_avg.loc[i,"SMOTE_va"] else 0 for i in range(len(df_best_scores_avg))]
df_best_scores_avg['(LS=SM)_va'] = [1 if df_best_scores_avg.loc[i,"LS_va"] == df_best_scores_avg.loc[i,"SMOTE_va"] else 0 for i in range(len(df_best_scores_avg))]
# Decision (test)
df_best_scores_avg['(LM>SM)_t'] = [1 if df_best_scores_avg.loc[i,"LLM_t"] > df_best_scores_avg.loc[i,"SMOTE_t"] else 0 for i in range(len(df_best_scores_avg))]
df_best_scores_avg['(LM=SM)_t'] = [1 if df_best_scores_avg.loc[i,"LLM_t"] == df_best_scores_avg.loc[i,"SMOTE_t"] else 0 for i in range(len(df_best_scores_avg))]
df_best_scores_avg['(LS>LM)_t'] = [1 if df_best_scores_avg.loc[i,"LS_t"] > df_best_scores_avg.loc[i,"LLM_t"] else 0 for i in range(len(df_best_scores_avg))]
df_best_scores_avg['(LS=LM)_t'] = [1 if df_best_scores_avg.loc[i,"LS_t"] == df_best_scores_avg.loc[i,"LLM_t"] else 0 for i in range(len(df_best_scores_avg))]
df_best_scores_avg['(LS>SM)_t'] = [1 if df_best_scores_avg.loc[i,"LS_t"] > df_best_scores_avg.loc[i,"SMOTE_t"] else 0 for i in range(len(df_best_scores_avg))]
df_best_scores_avg['(LS=SM)_t'] = [1 if df_best_scores_avg.loc[i,"LS_t"] == df_best_scores_avg.loc[i,"SMOTE_t"] else 0 for i in range(len(df_best_scores_avg))]

# Achievement Rate (AR_va/tr)
df_best_scores_avg['SM_va/SM_tr'] = df_best_scores_avg.loc[:,'SMOTE_va'].astype(float) / df_best_scores_avg.loc[:,'SMOTE_tr'].astype(float)
df_best_scores_avg['LM_va/LM_tr'] = df_best_scores_avg.loc[:,'LLM_va'].astype(float) / df_best_scores_avg.loc[:,'LLM_tr'].astype(float)
df_best_scores_avg['LS_va/LS_tr'] = df_best_scores_avg.loc[:,'LS_va'].astype(float) / df_best_scores_avg.loc[:,'LS_tr'].astype(float)
# Achievement Rate (AR_t/tr)
df_best_scores_avg['SM_t/SM_tr'] = df_best_scores_avg.loc[:,'SMOTE_t'].astype(float) / df_best_scores_avg.loc[:,'SMOTE_tr'].astype(float)
df_best_scores_avg['LM_t/LM_tr'] = df_best_scores_avg.loc[:,'LLM_t'].astype(float) / df_best_scores_avg.loc[:,'LLM_tr'].astype(float)
df_best_scores_avg['LS_t/LS_tr'] = df_best_scores_avg.loc[:,'LS_t'].astype(float) / df_best_scores_avg.loc[:,'LS_tr'].astype(float)
# Achievement Rate (AR_t/va)
df_best_scores_avg['SM_t/SM_va'] = df_best_scores_avg.loc[:,'SMOTE_t'].astype(float) / df_best_scores_avg.loc[:,'SMOTE_va'].astype(float)
df_best_scores_avg['LM_t/LM_va'] = df_best_scores_avg.loc[:,'LLM_t'].astype(float) / df_best_scores_avg.loc[:,'LLM_va'].astype(float)
df_best_scores_avg['LS_t/LS_va'] = df_best_scores_avg.loc[:,'LS_t'].astype(float) / df_best_scores_avg.loc[:,'LS_va'].astype(float)

df_best_scores_avg

## 1.1 Performance - SMOTE vs LLM vs LS

In [ ]:
# winning numbers (test)
print(sum(df_best_scores_avg['(LM>SM)_t']), sum(df_best_scores_avg['(LS>LM)_t']), sum(df_best_scores_avg['(LS>SM)_t']))
print(sum(df_best_scores_avg['(LM=SM)_t']), sum(df_best_scores_avg['(LS=LM)_t']), sum(df_best_scores_avg['(LS=SM)_t']))
print(len(df_best_scores_avg)-sum(df_best_scores_avg['(LM>SM)_t'])-sum(df_best_scores_avg['(LM=SM)_t']),
      len(df_best_scores_avg)-sum(df_best_scores_avg['(LS>LM)_t'])-sum(df_best_scores_avg['(LS=LM)_t']),
      len(df_best_scores_avg)-sum(df_best_scores_avg['(LS>SM)_t'])-sum(df_best_scores_avg['(LS=SM)_t']))

## 1.2 Performance Margin

In [ ]:
# test
print("ORG_t", np.average(df_best_scores_avg.loc[:,'ORG_t']))
print("SM_t", np.average(df_best_scores_avg.loc[:,'SMOTE_t']))
print("LM_t", np.average(df_best_scores_avg.loc[:,'LLM_t']))
print("LS_t", np.average(df_best_scores_avg.loc[:,'LS_t']))
print("avg. margin (LM-SM)_t:", np.average(df_best_scores_avg.loc[:,'(LM-SM)_t']))
print("avg. margin (LS-LM)_t:", np.average(df_best_scores_avg.loc[:,'(LS-LM)_t']))
print("avg. margin (LS-SM)_t:", np.average(df_best_scores_avg.loc[:,'(LS-SM)_t']))

## 1-3. Winning Probability

In [ ]:
rope = 0.01

In [ ]:
# test

In [ ]:
BST(rope, ['SMOTE_t'], ['LLM_t'], [df_best_scores_avg])  # BST(rope, baselines, ours, dfs)

In [ ]:
BST(rope, ['LLM_t'], ['LS_t'], [df_best_scores_avg])  # BST(rope, baselines, ours, dfs)

In [ ]:
BST(rope, ['SMOTE_t'], ['LS_t'], [df_best_scores_avg])  # BST(rope, baselines, ours, dfs)

## 2.1 Performance in Imbalance - SMOTE vs LLM vs LS

In [ ]:
half = int(len(final_list)/3)
half

## 2.2 Performance Margin in Imbalance (LLM-SMOTE) | (LLM-LS) | (LS-SMOTE)

In [ ]:
## test
# in more imbalanced
print("avg. margin (LM-SM)_t:", np.average(df_best_scores_avg.loc[:half-1,'(LM-SM)_t']))
# in mid imbalanced
print("avg. margin (LM-SM)_t:", np.average(df_best_scores_avg.loc[half:2*half-1,'(LM-SM)_t']))
# in less imbalanced
print("avg. margin (LM-SM)_t:", np.average(df_best_scores_avg.loc[2*half:,'(LM-SM)_t']))

# in more imbalanced
print("avg. margin (LS-LM)_t:", np.average(df_best_scores_avg.loc[:half-1,'(LS-LM)_t']))
# in mid imbalanced
print("avg. margin (LS-LM)_t:", np.average(df_best_scores_avg.loc[half:2*half-1,'(LS-LM)_t']))
# in less imbalanced
print("avg. margin (LS-LM)_t:", np.average(df_best_scores_avg.loc[2*half:,'(LS-LM)_t']))

# in more imbalanced
print("avg. margin (LS-SM)_t:", np.average(df_best_scores_avg.loc[:half-1,'(LS-SM)_t']))
# in mid imbalanced
print("avg. margin (LS-SM)_t:", np.average(df_best_scores_avg.loc[half:2*half-1,'(LS-SM)_t']))
# in less imbalanced
print("avg. margin (LS-SM)_t:", np.average(df_best_scores_avg.loc[2*half:,'(LS-SM)_t']))

## 2.3 Winning Probability

In [ ]:
rope = 0.01

In [ ]:
# in more imbalanced
BST(rope, ['SMOTE_t'], ['LLM_t'], [df_best_scores_avg.iloc[:half,:]])  # BST(rope, baselines, ours, dfs)

In [ ]:
# mid imbalanced
BST(rope, ['SMOTE_t'], ['LLM_t'], [df_best_scores_avg.iloc[half:-half,:]])  # BST(rope, baselines, ours, dfs)

In [ ]:
# in less imbalanced
BST(rope, ['SMOTE_t'], ['LLM_t'], [df_best_scores_avg.iloc[-half:,:]])  # BST(rope, baselines, ours, dfs)

In [ ]:
# in more imbalanced
BST(rope, ['LLM_t'], ['LS_t'], [df_best_scores_avg.iloc[:half,:]])  # BST(rope, baselines, ours, dfs)

In [ ]:
# mid imbalanced
BST(rope, ['LLM_t'], ['LS_t'], [df_best_scores_avg.iloc[half:-half,:]])  # BST(rope, baselines, ours, dfs)

In [ ]:
# in less imbalanced
BST(rope, ['LLM_t'], ['LS_t'], [df_best_scores_avg.iloc[-half:,:]])  # BST(rope, baselines, ours, dfs)

In [ ]:
# in more imbalanced
BST(rope, ['SMOTE_t'], ['LS_t'], [df_best_scores_avg.iloc[:half,:]])  # BST(rope, baselines, ours, dfs)

In [ ]:
# mid imbalanced
BST(rope, ['SMOTE_t'], ['LS_t'], [df_best_scores_avg.iloc[half:-half,:]])  # BST(rope, baselines, ours, dfs)

In [ ]:
# in less imbalanced
BST(rope, ['SMOTE_t'], ['LS_t'], [df_best_scores_avg.iloc[-half:,:]])  # BST(rope, baselines, ours, dfs)

## 2.4 Correlation Analysis with Margin (LLM-SMOTE) | (LLM-LS) | (LS-SMOTE)

In [ ]:
# plt.rcParams["figure.figsize"] = [5, 3]
plt.rcParams["figure.dpi"] = 200
plt.rcParams['axes.titlesize'] = 20  
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['axes.labelsize'] = 20  
plt.rcParams['font.size'] = 15

In [ ]:
# Visualization
for i in range(3,4):
    X = df_meta.iloc[:,i]   # imbalance ratio
    Y = df_best_scores_avg.loc[:,'(LM-SM)_t']  # 'Margin' Or 'Decision'
    plt.scatter(X, Y, alpha=0.5)
    plt.title('Correlation of (LM-SM) & IR')
    plt.xlabel('IR (X:1)')
    plt.ylabel('(LM-SM)')
    plt.show()
    
# chatGPT 추천
import pandas as pd
from scipy.stats import pearsonr, spearmanr

# Convert to pandas Series
imb = pd.Series(X)
ach = pd.Series(Y)

# Pearson correlation (linear)
pearson_corr, pearson_p = pearsonr(imb, ach)

# Spearman correlation (rank-based)
spearman_corr, spearman_p = spearmanr(imb, ach)

print("Pearson correlation:", pearson_corr, " (p =", pearson_p, ")")
print("Spearman correlation:", spearman_corr, " (p =", spearman_p, ")")

In [ ]:
# Visualization
for i in range(3,4):
    X = df_meta.iloc[:,i]   # imbalance ratio
    Y = df_best_scores_avg.loc[:,'(LS-LM)_t']  # 'Margin' Or 'Decision'
    plt.scatter(X, Y, alpha=0.5)
    plt.title('Correlation of (LS-LM) & IR')
    plt.xlabel('IR (X:1)')
    plt.ylabel('(LS-LM)')
    plt.show()
    
# chatGPT 추천
import pandas as pd
from scipy.stats import pearsonr, spearmanr

# Convert to pandas Series
imb = pd.Series(X)
ach = pd.Series(Y)

# Pearson correlation (linear)
pearson_corr, pearson_p = pearsonr(imb, ach)

# Spearman correlation (rank-based)
spearman_corr, spearman_p = spearmanr(imb, ach)

print("Pearson correlation:", pearson_corr, " (p =", pearson_p, ")")
print("Spearman correlation:", spearman_corr, " (p =", spearman_p, ")")

In [ ]:
# Visualization
for i in range(3,4):
    X = df_meta.iloc[:,i]   # imbalance ratio
    Y = df_best_scores_avg.loc[:,'(LS-SM)_t']  # 'Margin' Or 'Decision'
    plt.scatter(X, Y, alpha=0.5)
    plt.title('Correlation of (LS-SM) & IR')
    plt.xlabel('IR (X:1)')
    plt.ylabel('(LS-SM)')
    plt.show()
    
# chatGPT 추천
import pandas as pd
from scipy.stats import pearsonr, spearmanr

# Convert to pandas Series
imb = pd.Series(X)
ach = pd.Series(Y)

# Pearson correlation (linear)
pearson_corr, pearson_p = pearsonr(imb, ach)

# Spearman correlation (rank-based)
spearman_corr, spearman_p = spearmanr(imb, ach)

print("Pearson correlation:", pearson_corr, " (p =", pearson_p, ")")
print("Spearman correlation:", spearman_corr, " (p =", spearman_p, ")")

## 3.1 Overfitting - Achievement Rate of SMOTE vs LLM vs LS

In [ ]:
df_best_scores_avg.loc[:,['DATA',
                          'SM_va/SM_tr','LM_va/LM_tr','LS_va/LS_tr',
                          'SM_t/SM_tr','LM_t/LM_tr','LS_t/LS_tr',
                          'SM_t/SM_va','LM_t/LM_va','LS_t/LS_va']]   
# if <1 overfitting, smaller->more overfitting
# especially, if it significantly >1 or <1, overfitting and underfitting
# 각 데이터별로 비교가 좋을 듯

In [ ]:
# testing/validation
print("SM", np.average(df_best_scores_avg.loc[:,"SM_t/SM_va"]), np.std(df_best_scores_avg.loc[:,"SM_t/SM_va"]))
print("LM", np.average(df_best_scores_avg.loc[:,"LM_t/LM_va"]), np.std(df_best_scores_avg.loc[:,"LM_t/LM_va"]))
print("LS", np.average(df_best_scores_avg.loc[:,"LS_t/LS_va"]), np.std(df_best_scores_avg.loc[:,"LS_t/LS_va"]))

print("SM", np.var(df_best_scores_avg.loc[:,"SM_t/SM_va"]))
print("LM", np.var(df_best_scores_avg.loc[:,"LM_t/LM_va"]))
print("LS", np.var(df_best_scores_avg.loc[:,"LS_t/LS_va"]))

In [ ]:
# Visualization
for i in range(3,4):
    X = df_meta.iloc[:,i]   # imbalance ratio
    Y = df_best_scores_avg.loc[:,"SM_t/SM_va"]  # Achievment Rate
    plt.scatter(X, Y, alpha=0.5)
    plt.title('Correlation of (Test/Val)_SM & IR')
    plt.xlabel('IR (X:1)')
    plt.ylabel('(Test/Val)_SM')
    plt.show()
    
import pandas as pd
from scipy.stats import pearsonr, spearmanr

# Convert to pandas Series
imb = pd.Series(X)
ach = pd.Series(Y)

# Pearson correlation (linear)
pearson_corr, pearson_p = pearsonr(imb, ach)

# Spearman correlation (rank-based)
spearman_corr, spearman_p = spearmanr(imb, ach)

print("Pearson correlation:", pearson_corr, " (p =", pearson_p, ")")
print("Spearman correlation:", spearman_corr, " (p =", spearman_p, ")")

In [ ]:
# Visualization
for i in range(3,4):
    X = df_meta.iloc[:,i]   # imbalance ratio
    Y = df_best_scores_avg.loc[:,"LM_t/LM_va"]  # Achievment Rate
    plt.scatter(X, Y, alpha=0.5)
    plt.title('Correlation of (Test/Val)_LM & IR')
    plt.xlabel('IR (X:1)')
    plt.ylabel('(Test/Val)_LM')
    plt.show()
    
import pandas as pd
from scipy.stats import pearsonr, spearmanr

# Convert to pandas Series
imb = pd.Series(X)
ach = pd.Series(Y)

# Pearson correlation (linear)
pearson_corr, pearson_p = pearsonr(imb, ach)

# Spearman correlation (rank-based)
spearman_corr, spearman_p = spearmanr(imb, ach)

print("Pearson correlation:", pearson_corr, " (p =", pearson_p, ")")
print("Spearman correlation:", spearman_corr, " (p =", spearman_p, ")")

In [ ]:
# Visualization
for i in range(3,4):
    X = df_meta.iloc[:,i]   # imbalance ratio
    Y = df_best_scores_avg.loc[:,"LS_t/LS_va"]  # Achievment Rate
    plt.scatter(X, Y, alpha=0.5)
    plt.title('Correlation of (Test/Val)_LS & IR')
    plt.xlabel('IR (X:1)')
    plt.ylabel('(Test/Val)_LS')
    plt.ylim(-0.05, 1.25)
    plt.show()
    
import pandas as pd
from scipy.stats import pearsonr, spearmanr

# Convert to pandas Series
imb = pd.Series(X)
ach = pd.Series(Y)

# Pearson correlation (linear)
pearson_corr, pearson_p = pearsonr(imb, ach)

# Spearman correlation (rank-based)
spearman_corr, spearman_p = spearmanr(imb, ach)

print("Pearson correlation:", pearson_corr, " (p =", pearson_p, ")")
print("Spearman correlation:", spearman_corr, " (p =", spearman_p, ")")